In [8]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
_ = {
    "univariate": ['energy (kw 15min)'],
}

# Input Report Simulation
IRS = []
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )

# Training Model Array 
TMA = []

is_switch = False
compare_df = pd.DataFrame(columns=['non-cluster mae', 'cluster mae', 'winner'])

for _ in range(0, 100):
    MAE_LIST = []
    for _ in IRS:
        print("--- now info ---")

        bm = BasicModel(name=_.name,
                       columns=_.columns,
                       is_contain_cluster_label=_.is_label,
                       is_switch=is_switch,
                       matching_type=_.matching,
                       is_multi_step=True,
                       model_type="Multi Step / SingleShot",
                       clustering_type="new")
        bm.set_window(OUT_STEPS=3)
        bm.set_model()
        print(bm)
        print(compare_df)
        bm.training(epochs=50)

        bm.set_multi_predict(is_reshape=True)
        
        statistic = bm.statistic()
        
        MAE_LIST.append(statistic['mae'])
        
    winner = "non-cluster" if MAE_LIST[0] < MAE_LIST[1] else \
        "draw" if MAE_LIST[0] == MAE_LIST[1] else "with-cluster"
    compare_df = compare_df.append({
        "non-cluster mae": MAE_LIST[0],
        "cluster mae": MAE_LIST[1],
        "winner": winner
    }, ignore_index=True)

###### [Notice] (univariate / with cluster) set predict (test) info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 



In [13]:
compare_df[compare_df['winner'] == 'non-cluster']

,non-cluster mae,cluster mae,winner


In [14]:
compare_df[compare_df['winner'] == 'with-cluster']

,non-cluster mae,cluster mae,winner
0,0.278334,0.270114,with-cluster
1,0.276872,0.269604,with-cluster
2,0.277157,0.268861,with-cluster
3,0.277647,0.271763,with-cluster
4,0.277279,0.267365,with-cluster
...,...,...,...
95,0.277942,0.268895,with-cluster
96,0.278280,0.267552,with-cluster
97,0.277200,0.265161,with-cluster
98,0.277366,0.269099,with-cluster
